In [8]:
import pandas as pd
import numpy as np
from pathlib import Path
import nltk, spacy
from nltk.corpus import stopwords
from wordcloud import STOPWORDS
from collections import Counter
from gensim.parsing.preprocessing import STOPWORDS as SW

pd.options.display.float_format = '{:,.0f}'.format

In [9]:
in_file = Path.cwd().parents[0] / 'Processed_datasets' / 'tweets_long_no_duplicates.csv'
my_stopwords = stopwords.words('english')
stopwords = set(STOPWORDS).union(my_stopwords) #preparing stopwards list
custom_stopwords = ['hi', '\n', '\n\n', '&amp;', ' ', '.', '-',
                    'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']
nlp = spacy.load('en_core_web_sm', exclude=['ner', 'tok2vec', 'tagger', 'paerser', 'senter', 'lemmatizer', 'attribute_ruler']) # using only for stopwords
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)

In [10]:
df_long = pd.read_csv(in_file, engine='python', usecols=['Tokens', 'Label']) #not using unnecessary columns

In [11]:
print(df_long.head(15))
df_long.count()
print(df_long['Label'].value_counts())

                                               Tokens  Label
0   muslim mob violence against hindus in banglade...      1
1   islamophobia is like the idea of naziphobia is...      1
2   finally all caught up and that sudden death co...      0
3   please please start using is your discernment ...      0
4   as soon as isis chased all the minorities out ...      0
5   islam invaded and conquered of christiandom be...      1
6   do you approve of your pedophile prophet rapin...      1
7   problem with vile muslims is that they try to ...      1
8            tend to talk about it much personal info      0
9   cool next time when a woman talks to him about...      0
10  our judges are about to turn the heat up in th...      0
11  lol this you walk by putting one foot in front...      0
12  said wanted sorbet now and they should tell us...      0
13  this fucking potato is blowing my mind duck fa...      0
14  omg this churner feels like razor blades on my...      0
0    15214
1     7500
Na

In [12]:
#setting up baseline pipeline
from nltk.tokenize import TweetTokenizer #I chose to tokenize with this, as it gets rid of @ handlers
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler


In [13]:
def make_dataset(d_frame, stopwords=True, tfidf=True, train=True): #function for creating dataset, paramaters for flexibility
    """Helper function for construction the classes later on - accepts a dataframe 
    as an input and 3 boolean and ouputs a tuple of either the test or dev set
    The code block essentially repeates itself with minor argument tweaks
    When stopwords=True then stopwords are removed, when tfidf=True then tfidf     is implemented and when train-True then train set is returned"""
    df = d_frame
    if stopwords and tfidf and train:                                                       #all included
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = TfidfVectorizer(max_df=.9, min_df=25, strip_accents='unicode',
                               tokenizer=tknzr.tokenize, stop_words=ALL_STOP_WORDS)
        sampler_train = RandomUnderSampler(random_state=42) #undersampling  object to rebalance dataset
        xs, ys = vect.fit_transform(df['Tokens'].values), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85, #creating train and dev sets with stratified sampling
                                                          random_state=42, stratify=ys) 
        x_train, y_train = sampler_train.fit_resample(x_train, y_train) 
        return x_train, y_train
    elif not stopwords and tfidf and train: #no stopwords
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = TfidfVectorizer(max_df=.9, min_df=25,strip_accents='unicode',
                               tokenizer=tknzr.tokenize)
        sampler_train = RandomUnderSampler(random_state=42) 
        xs, ys = vect.fit_transform(df['Tokens'].values), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85,
                                                          random_state=42, stratify=ys)
        x_train, y_train = sampler_train.fit_resample(x_train, y_train) 
        return x_train, y_train    
    elif stopwords and not tfidf and train: #no tfidf
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = CountVectorizer(max_df=.9, min_df=25,
                       strip_accents='unicode', tokenizer=tknzr.tokenize, stop_words=ALL_STOP_WORDS)
        sampler_train = RandomUnderSampler(random_state=42) 
        xs, ys = vect.fit_transform(df['Tokens'].values), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85,
                                                          random_state=42, stratify=ys)
        x_train, y_train = sampler_train.fit_resample(x_train, y_train) 
        return x_train, y_train
    elif not stopwords and not tfidf and train:
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = CountVectorizer(max_df=.9, min_df=25,
                       strip_accents='unicode', tokenizer=tknzr.tokenize)
        sampler_train = RandomUnderSampler(random_state=42) 
        xs, ys = vect.fit_transform(df['Tokens'].values), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85,
                                                          random_state=42, stratify=ys)
        x_train, y_train = sampler_train.fit_resample(x_train, y_train) 
        return x_train, y_train
    elif stopwords and tfidf and not train:
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = TfidfVectorizer(max_df=.9, min_df=25,
                       strip_accents='unicode', tokenizer=tknzr.tokenize, stop_words=ALL_STOP_WORDS)
        sampler_dev = RandomUnderSampler(random_state=50) 
        xs, ys = vect.fit_transform(df['Tokens'].values), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85,
                                                          random_state=42, stratify=ys)
        x_dev, y_dev = sampler_dev.fit_resample(x_dev, y_dev)
        return x_dev, y_dev
    elif not stopwords and tfidf and not train:
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = TfidfVectorizer(max_df=.9, min_df=25,strip_accents='unicode',
                               tokenizer=tknzr.tokenize)
        sampler_dev = RandomUnderSampler(random_state=50) 
        xs, ys = vect.fit_transform(df['Tokens'].values), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85,
                                                          random_state=42, stratify=ys)
        x_dev, y_dev = sampler_dev.fit_resample(x_dev, y_dev)
        return x_dev, y_dev
    elif stopwords and not tfidf and not train:
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = CountVectorizer(max_df=.9, min_df=25,strip_accents='unicode',
                               tokenizer=tknzr.tokenize, stop_words=ALL_STOP_WORDS)
        sampler_dev = RandomUnderSampler(random_state=50) 
        xs, ys = vect.fit_transform(df['Tokens'].values), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85,
                                                          random_state=42, stratify=ys)
        x_dev, y_dev = sampler_dev.fit_resample(x_dev, y_dev)
        return x_dev, y_dev
    else:
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = CountVectorizer(max_df=.9, min_df=25,strip_accents='unicode',
                               tokenizer=tknzr.tokenize)
        sampler_dev = RandomUnderSampler(random_state=50) 
        xs, ys = vect.fit_transform(df_long['Tokens'].values), df_long['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85,
                                                          random_state=42, stratify=ys)
        x_dev, y_dev = sampler_dev.fit_resample(x_dev, y_dev)
        return x_dev, y_dev    


        
def make_scaled_dataset(df, stopwords=True, tfidf=True, train=True):
    '''Same as above function except with added functionality of StandardScalar, creating function with 5 options was confusing'''
    if stopwords and tfidf and train:
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = TfidfVectorizer(max_df=.9, min_df=25, strip_accents='unicode',
                               tokenizer=tknzr.tokenize, stop_words=ALL_STOP_WORDS)
        sampler_train = RandomUnderSampler(random_state=42) #undersampling  object to rebalance dataset
        scalar = StandardScaler(with_mean=False)
        xs, ys = scalar.fit_transform(vect.fit_transform(df['Tokens'].values)), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85, #creating train and dev sets with stratified sampling
                                                          random_state=42, stratify=ys) 
        x_train, y_train = sampler_train.fit_resample(x_train, y_train) 
        return x_train, y_train
    elif stopwords and tfidf and not train:
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = TfidfVectorizer(max_df=.9, min_df=25, strip_accents='unicode',
                               tokenizer=tknzr.tokenize, stop_words=ALL_STOP_WORDS)
        sampler_train = RandomUnderSampler(random_state=42) #undersampling  object to rebalance dataset
        scalar = StandardScaler(with_mean=False)
        xs, ys = scalar.fit_transform(vect.fit_transform(df['Tokens'].values)), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85, #creating train and dev sets with stratified sampling
                                                          random_state=42, stratify=ys) 
        x_dev, y_dev = sampler_train.fit_resample(x_dev, y_dev) 
        return x_dev, y_dev
    elif not stopwords and tfidf and train:
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = TfidfVectorizer(max_df=.9, min_df=25, strip_accents='unicode',
                               tokenizer=tknzr.tokenize)
        sampler_train = RandomUnderSampler(random_state=42) #undersampling  object to rebalance dataset
        scalar = StandardScaler(with_mean=False)
        xs, ys = scalar.fit_transform(vect.fit_transform(df['Tokens'].values)), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85, #creating train and dev sets with stratified sampling
                                                          random_state=42, stratify=ys) 
        x_train, y_train = sampler_train.fit_resample(x_train, y_train) 
        return x_train, y_train
    elif not stopwords and tfidf and not train:
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = TfidfVectorizer(max_df=.9, min_df=25, strip_accents='unicode',
                               tokenizer=tknzr.tokenize)
        sampler_train = RandomUnderSampler(random_state=42) #undersampling  object to rebalance dataset
        scalar = StandardScaler(with_mean=False)
        xs, ys = scalar.fit_transform(vect.fit_transform(df['Tokens'].values)), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85, #creating train and dev sets with stratified sampling
                                                          random_state=42, stratify=ys) 
        x_dev, y_dev = sampler_train.fit_resample(x_dev, y_dev) 
        return x_dev, y_dev
    elif stopwords and not tfidf and train:
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = CountVectorizer(max_df=.9, min_df=25, strip_accents='unicode',
                               tokenizer=tknzr.tokenize, stop_words=ALL_STOP_WORDS)
        sampler_train = RandomUnderSampler(random_state=42) #undersampling  object to rebalance dataset
        scalar = StandardScaler(with_mean=False)
        xs, ys = scalar.fit_transform(vect.fit_transform(df['Tokens'].values)), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85, #creating train and dev sets with stratified sampling
                                                          random_state=42, stratify=ys) 
        x_train, y_train = sampler_train.fit_resample(x_train, y_train) 
        return x_train, y_train
    elif stopwords and not tfidf and not train:
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = CountVectorizer(max_df=.9, min_df=25, strip_accents='unicode',
                               tokenizer=tknzr.tokenize, stop_words=ALL_STOP_WORDS)
        sampler_train = RandomUnderSampler(random_state=42) #undersampling  object to rebalance dataset
        scalar = StandardScaler(with_mean=False)
        xs, ys = scalar.fit_transform(vect.fit_transform(df['Tokens'].values)), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85, #creating train and dev sets with stratified sampling
                                                          random_state=42, stratify=ys) 
        x_dev, y_dev = sampler_train.fit_resample(x_dev, y_dev) 
        return x_dev, y_dev
    elif not stopwords and not tfidf and train:
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = CountVectorizer(max_df=.9, min_df=25, strip_accents='unicode',
                               tokenizer=tknzr.tokenize)
        sampler_train = RandomUnderSampler(random_state=42) #undersampling  object to rebalance dataset
        scalar = StandardScaler(with_mean=False)
        xs, ys = scalar.fit_transform(vect.fit_transform(df['Tokens'].values)), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85, #creating train and dev sets with stratified sampling
                                                          random_state=42, stratify=ys) 
        x_train, y_train = sampler_train.fit_resample(x_train, y_train) 
        return x_train, y_train
    elif not stopwords and not tfidf and not train:
        tknzr = TweetTokenizer(preserve_case=True, reduce_len=True)
        vect = CountVectorizer(max_df=.9, min_df=25, strip_accents='unicode',
                               tokenizer=tknzr.tokenize)
        sampler_train = RandomUnderSampler(random_state=42) #undersampling  object to rebalance dataset
        scalar = StandardScaler(with_mean=False)
        xs, ys = scalar.fit_transform(vect.fit_transform(df['Tokens'].values)), df['Label'].values
        x_train, x_dev, y_train, y_dev = train_test_split(xs, ys, train_size=.85, #creating train and dev sets with stratified sampling
                                                          random_state=42, stratify=ys) 
        x_dev, y_dev = sampler_train.fit_resample(x_dev, y_dev) 
        return x_dev, y_dev

def make_dataset_iterators(df):
    """Creates a tuple of generators. First index holds all train sets while second holds all dev sets.
        Makes running the multiple datasets through a model more convenient """
    all_param_train = make_dataset(df)  # non scaled variation of the dataset
    # dev set counterpart of above
    all_param_dev = make_dataset(df, train=False)
    stopwords_no_tfidf_train = make_dataset(df, tfidf=False)
    stopwords_no_tfidf_dev = make_dataset(
        df, tfidf=False, train=False)  # devset counterpart of above
    no_stop_train = make_dataset(df, stopwords=False)
    no_stop_dev = make_dataset(df, stopwords=False, train=False)
    no_stopwords_no_tfidf_train = make_dataset(
        df, stopwords=False, tfidf=False)
    no_stopwords_no_tfidf_dev = make_dataset(
        df, stopwords=False, tfidf=False, train=False)

    scaled_all_param_train = make_scaled_dataset(df) 
    scaled_all_param_dev = make_scaled_dataset(df, train=False)
    scaled_stop_no_tfidf_train = make_scaled_dataset(df, tfidf=False)
    scaled_stop_no_tfidf_dev = make_scaled_dataset(
        df, tfidf=False, train=False)
    scaled_no_stop_tfidf_train = make_scaled_dataset(df, stopwords=False)
    scaled_no_stop_tfidf_dev = make_dataset(df, stopwords=False, train=False)
    scaled_no_stop_no_tfidf_train = make_dataset(
        df, stopwords=False, tfidf=False)
    scaled_no_stop_no_tfidf_dev = make_dataset(
        df, stopwords=False, tfidf=False, train=False)

    list_train_sets = [all_param_train, stopwords_no_tfidf_train, no_stop_train, no_stopwords_no_tfidf_train, scaled_all_param_train,       #make lists of previously created train                                                                                                                                                  datasets
                       scaled_stop_no_tfidf_train, scaled_no_stop_no_tfidf_train, scaled_no_stop_tfidf_train, scaled_no_stop_no_tfidf_train] # make lists of previously created dev sets
    list_dev_sets = [all_param_dev, stopwords_no_tfidf_dev, no_stop_dev, no_stopwords_no_tfidf_dev, scaled_all_param_dev,
                     scaled_stop_no_tfidf_dev, scaled_no_stop_no_tfidf_dev, scaled_no_stop_tfidf_dev, scaled_no_stop_no_tfidf_dev]

    train_gen, dev_gen = iter(list_train_sets), iter(list_dev_sets) #convert them both to iterators to save space 
    return train_gen, dev_gen





In [25]:
tuple_of_generators = make_dataset_iterators(df_long)
print(type(tuple_of_generators))
print(tuple_of_generators[0])

/Users/chengyiyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['&', "'", 'n', '‘', '’'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/chengyiyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['&', "'", 'n', '‘', '’'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/chengyiyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['&', "'", 'n', '‘', '’'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/Users/chengyiyang/opt/anaconda

<class 'tuple'>


In [18]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report

In [16]:
class ClassifierBot():
    
    def __init__(self, trainset, devset, bot):
        self.train = trainset
        self.dev = devset
        self.bot = bot
        
    def classify(self):
        self.bot.fit(self.train[0], self.train[1])
        self.prediction = self.bot.predict(self.dev[0])
        return classification_report(self.dev[1], self.prediction, output_dict=True)
        


In [2]:
def bots(iter_train, iter_dev, clf):
    bots = []
    preds = []
    for i in range(0, 9):
        bot = ClassifierBot(next(iter_train), next(iter_dev), clf)
        prediction = bot.classify()
        print(prediction)
        print(type(prediction))
        bots.append(bot)
        preds.append(preds)
    return bots, preds


In [26]:
tup_bots_sgd = bots(tuple_of_generators[0], tuple_of_generators[1], SGDClassifier(random_state=42))

{'0': {'precision': 0.7259430331023865, 'recall': 0.8382222222222222, 'f1-score': 0.778052805280528, 'support': 1125}, '1': {'precision': 0.8086225026288117, 'recall': 0.6835555555555556, 'f1-score': 0.7408477842003853, 'support': 1125}, 'accuracy': 0.7608888888888888, 'macro avg': {'precision': 0.7672827678655991, 'recall': 0.760888888888889, 'f1-score': 0.7594502947404567, 'support': 2250}, 'weighted avg': {'precision': 0.7672827678655991, 'recall': 0.7608888888888888, 'f1-score': 0.7594502947404567, 'support': 2250}}
<class 'dict'>
{'0': {'precision': 0.7126168224299065, 'recall': 0.8133333333333334, 'f1-score': 0.759651307596513, 'support': 1125}, '1': {'precision': 0.782608695652174, 'recall': 0.672, 'f1-score': 0.7230989956958394, 'support': 1125}, 'accuracy': 0.7426666666666667, 'macro avg': {'precision': 0.7476127590410402, 'recall': 0.7426666666666667, 'f1-score': 0.7413751516461762, 'support': 2250}, 'weighted avg': {'precision': 0.7476127590410403, 'recall': 0.74266666666666

/Users/chengyiyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
#Neural Network Classifier 
from sklearn.neural_network import MLPClassifier


In [16]:
tup_bots_MLP = bots(tuple_of_generators[0], tuple_of_generators[1], MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(10,30,10,5), random_state=42, batch_size=128, max_iter=1000))

{'0': {'precision': 0.6751760563380281, 'recall': 0.6817777777777778, 'f1-score': 0.6784608580274214, 'support': 1125}, '1': {'precision': 0.6786355475763016, 'recall': 0.672, 'f1-score': 0.6753014738722645, 'support': 1125}, 'accuracy': 0.6768888888888889, 'macro avg': {'precision': 0.6769058019571649, 'recall': 0.6768888888888889, 'f1-score': 0.6768811659498429, 'support': 2250}, 'weighted avg': {'precision': 0.6769058019571649, 'recall': 0.6768888888888889, 'f1-score': 0.676881165949843, 'support': 2250}}
<class 'dict'>
{'0': {'precision': 0.673557278208441, 'recall': 0.6951111111111111, 'f1-score': 0.68416447944007, 'support': 1125}, '1': {'precision': 0.6850321395775941, 'recall': 0.6631111111111111, 'f1-score': 0.6738934056007226, 'support': 1125}, 'accuracy': 0.6791111111111111, 'macro avg': {'precision': 0.6792947088930176, 'recall': 0.6791111111111111, 'f1-score': 0.6790289425203964, 'support': 2250}, 'weighted avg': {'precision': 0.6792947088930176, 'recall': 0.67911111111111

/Users/chengyiyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 1125}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1125}, 'accuracy': 0.5, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}}
<class 'dict'>


/Users/chengyiyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 1125}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1125}, 'accuracy': 0.5, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}}
<class 'dict'>
{'0': {'precision': 0.6605902777777778, 'recall': 0.6764444444444444, 'f1-score': 0.668423364075538, 'support': 1125}, '1': {'precision': 0.668488160291439, 'recall': 0.6524444444444445, 'f1-score': 0.6603688708951867, 'support': 1125}, 'accuracy': 0.6644444444444444, 'macro avg': {'precision': 0.6645392190346084, 'recall': 0.6644444444444444, 'f1-score': 0.6643961174853623, 'support': 2250}, 'weighted avg': {'precision': 0.6645392190346083, 'recall': 0.6644444444444444, 'f1-score': 0.6643961174853624, 'support': 2250}}
<class 'dict'>
{'0': {'precision': 0.6611927398444253, 'recall': 0.68, 'f1-scor

/Users/chengyiyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 1125}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1125}, 'accuracy': 0.5, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}}
<class 'dict'>


/Users/chengyiyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1125}, '1': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 1125}, 'accuracy': 0.5, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}}
<class 'dict'>
{'0': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 1125}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1125}, 'accuracy': 0.5, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}}
<class 'dict'>


/Users/chengyiyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
#Support Vector Classifier without Gridsearch
from sklearn.svm import SVC

tup_bots_svc = bots(tuple_of_generators[0], tuple_of_generators[1], SVC())


{'0': {'precision': 0.7183863460046548, 'recall': 0.8231111111111111, 'f1-score': 0.767191383595692, 'support': 1125}, '1': {'precision': 0.7929240374609782, 'recall': 0.6773333333333333, 'f1-score': 0.7305848513902207, 'support': 1125}, 'accuracy': 0.7502222222222222, 'macro avg': {'precision': 0.7556551917328165, 'recall': 0.7502222222222222, 'f1-score': 0.7488881174929563, 'support': 2250}, 'weighted avg': {'precision': 0.7556551917328165, 'recall': 0.7502222222222222, 'f1-score': 0.7488881174929563, 'support': 2250}}
<class 'dict'>
{'0': {'precision': 0.7123893805309734, 'recall': 0.8586666666666667, 'f1-score': 0.7787182587666264, 'support': 1125}, '1': {'precision': 0.8221476510067114, 'recall': 0.6533333333333333, 'f1-score': 0.7280832095096582, 'support': 1125}, 'accuracy': 0.756, 'macro avg': {'precision': 0.7672685157688424, 'recall': 0.756, 'f1-score': 0.7534007341381423, 'support': 2250}, 'weighted avg': {'precision': 0.7672685157688424, 'recall': 0.756, 'f1-score': 0.75340

/Users/chengyiyang/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 0.5, 'recall': 1.0, 'f1-score': 0.6666666666666666, 'support': 1125}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1125}, 'accuracy': 0.5, 'macro avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}, 'weighted avg': {'precision': 0.25, 'recall': 0.5, 'f1-score': 0.3333333333333333, 'support': 2250}}
<class 'dict'>
{'0': {'precision': 0.7105064247921391, 'recall': 0.8355555555555556, 'f1-score': 0.7679738562091503, 'support': 1125}, '1': {'precision': 0.8004314994606256, 'recall': 0.6595555555555556, 'f1-score': 0.7231968810916178, 'support': 1125}, 'accuracy': 0.7475555555555555, 'macro avg': {'precision': 0.7554689621263824, 'recall': 0.7475555555555555, 'f1-score': 0.745585368650384, 'support': 2250}, 'weighted avg': {'precision': 0.7554689621263824, 'recall': 0.7475555555555555, 'f1-score': 0.745585368650384, 'support': 2250}}
<class 'dict'>


In [24]:
#RandomForestClassifier without Gridsearch

from sklearn.ensemble import RandomForestClassifier


tup_bots_forest = bots(tuple_of_generators[0], tuple_of_generators[1], RandomForestClassifier(n_jobs=-1, random_state=42))

{'0': {'precision': 0.7456896551724138, 'recall': 0.7688888888888888, 'f1-score': 0.7571115973741794, 'support': 1125}, '1': {'precision': 0.7614678899082569, 'recall': 0.7377777777777778, 'f1-score': 0.7494356659142213, 'support': 1125}, 'accuracy': 0.7533333333333333, 'macro avg': {'precision': 0.7535787725403353, 'recall': 0.7533333333333333, 'f1-score': 0.7532736316442004, 'support': 2250}, 'weighted avg': {'precision': 0.7535787725403355, 'recall': 0.7533333333333333, 'f1-score': 0.7532736316442004, 'support': 2250}}
<class 'dict'>
{'0': {'precision': 0.73502722323049, 'recall': 0.72, 'f1-score': 0.7274360125729681, 'support': 1125}, '1': {'precision': 0.725609756097561, 'recall': 0.7404444444444445, 'f1-score': 0.7329520457545093, 'support': 1125}, 'accuracy': 0.7302222222222222, 'macro avg': {'precision': 0.7303184896640255, 'recall': 0.7302222222222222, 'f1-score': 0.7301940291637388, 'support': 2250}, 'weighted avg': {'precision': 0.7303184896640255, 'recall': 0.73022222222222

In [46]:
#classification report
from sklearn.metrics import accuracy_score,recall_score,precision_score,recall_score,f1_score
import warnings
warnings.filterwarnings('ignore')

def report(prediction, real_labels, data):
    print("classification report as follows: ")
    print(f'   Accuracy: {accuracy_score(prediction, real_labels)}')
    print(f'   Precision: {precision_score(prediction, real_labels,average="macro")}')
    print(f'   recall: {recall_score(prediction, real_labels,average="macro")}')
    print(f'   F1 measure: {f1_score(prediction, real_labels,average="macro")}')
    print('Show 5 example of correctly classified datapoint: ')
    if data[prediction==real_labels].shape[0] > 5:
        display(data[prediction==real_labels].iloc[:5,:])
    else:
        display(data[prediction==real_labels])
    print('Show 5 example of wrongly classified datapoint: ')
    if data[prediction!=real_labels].shape[0] > 5:
        display(data[prediction!=real_labels].iloc[:5,:])
    else:
        display(data[prediction!=real_labels])

In [132]:
val_x = vectorizer.transform(validation_set['Tokens'])
val_x = transformer.transform(val_x)

predict = ntwk.predict(val_x)
report(predict, validation_set['label'], validation_set[['Tokens','Label']])

classification report as follows: 
   Accuracy: 0.8168943476626144
   Precision: 0.7084592624109877
   recall: 0.7811589138333501
   F1 measure: 0.7309291098045785
Show 5 example of correctly classified datapoint: 


,Tokens,Label
23121,rt strategic vote kat food truly awful #mkr,0
35928,pancakes proof deity love us #mkr,0
45316,sick see fuck asshole bitch make chain latters...,1
42230,i'm try get insight trans issue definitely gro...,0
40128,make,0


Show 5 example of wrongly classified datapoint: 


,Tokens,Label
48658,lol ralph guy still moi era,1
30437,kill muslims oppose kill ezidis christians non...,0
28396,single men cannot adopt,1
44924,muslim brotherhood usa hundred years liken say...,0
7346,rt #mosul christian pastor #paul_jacob sentenc...,1
